In [1]:
import carla
import time
import sys

sys.path.append("../../carla_utils/utils")
sys.path.append("../../grasp_path_planner/scripts")


from intersection_scenario_manager import IntersectionScenario
import numpy as np
sys.path.append("cartesian_to_frenet/")
from topology_extraction import get_opendrive_tree, get_junction_topology, get_junction_roads_topology
sys.path.append("../../global_route_planner/")
from global_planner import get_client, draw_waypoints, spawn_vehicle

import shapely
from shapely.geometry import Point, LineString, MultiLineString


In [2]:
# Get client to interact with CARLA server
client = get_client()

# Get current CARLA world
world = client.get_world()

all_waypoints = world.get_map().generate_waypoints(1)
def draw(road_id, lane_id=None, color=[0,255,0]):
    for wp in all_waypoints:
        
        if lane_id == None and wp.road_id == road_id:
            draw_waypoints(world, [wp], 3, color) 
#             print(wp.lane_id)
        elif wp.road_id == road_id and wp.lane_id==lane_id:
            draw_waypoints(world, [wp], 3, color) 
            
def filter_waypoints(waypoints, road_id, lane_id = None):
    
    filtered_waypoints = []
    for wp in waypoints:
        
        if lane_id == None and wp.road_id == road_id:
            filtered_waypoints.append(wp) 
        elif wp.road_id == road_id and wp.lane_id==lane_id:
            filtered_waypoints.append(wp) 

    return filtered_waypoints
            
tree = get_opendrive_tree(world)
junction_topology = get_junction_topology(tree)
road_topology = get_junction_roads_topology(tree)


Connection to CARLA server established!


In [3]:
road_topology

{62: (4,
  5,
  [['backward', 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
   ['backward', 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]),
 63: (4,
  5,
  [['forward', -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
   ['forward', -2, -2, -2, -2, -2, -2, -2, -2, -2, -2]]),
 80: (4, 7, [['forward', -1, -1, -1, -1, -1, 1]]),
 84: (4, 7, [['backward', 2, 2, 2, 2, 1, -2]]),
 88: (4, 8, [['backward', 1, 1, 1, 1, 1, 1]]),
 92: (4, 8, [['forward', -2, -1, -2, -2, -2, -2]]),
 96: (5, 7, [['backward', -1, 1, 1, 1, 1, -1]]),
 100: (5, 7, [['forward', 2, -1, -2, -2, -2, 2]]),
 104: (5, 8, [['forward', 1, -1, -1, -1, -1, -1]]),
 108: (5, 8, [['backward', -2, 2, 2, 2, 1, 2]]),
 125: (7,
  8,
  [['backward', 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
   ['backward', 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]),
 126: (7,
  8,
  [['forward', -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
   ['forward', -2, -2, -2, -2, -2, -2, -2, -2, -2, -2, -2]]),
 140: (41, 21, [['backward', -1, 1, 1, 1, -1]]),
 143: (41, 21, [['forward', 2, -2, -2, -2, -1, -2, -2, -2, 2

In [4]:
draw(21,-1)
draw(207,-1)

In [5]:
SM = IntersectionScenario(client)

Intersection Manager Initialized...


dict_keys([224, 965, 421, 1175, 905, 1162, 139, 1260, 685, 334, 751, 1148, 1050, 53, 599, 1070, 943, 509, 924, 829, 245])

In [77]:
junctionId = 751

In [79]:
ego_vehicle, my_vehicles, incoming_road_lane_id_to_outgoing_lane_id_dict, intersection_topology, ego_key, global_path_wps, road_lane_to_orientation = SM.reset(num_vehicles=0, warm_start_duration=1, junction_id=junctionId)

generating scenario at junction id:  751 Num vehicles: 0
Control handed to system....


In [83]:
for elem in intersection_topology:
    for path in elem:
        for k in path:
            if(not isinstance(k, int)):
                draw(*k)


In [78]:
ego_vehicle.destroy()
for actor in my_vehicles:
    actor.destroy()

In [25]:
draw_waypoints(world, global_path_wps)

In [22]:
intersecting_left, intersecting_right, parallel_same_dir, parallel_opposite_dir = intersection_topology
ego_lane, parallel_same_dir = separate_ego_lane_and_other_parallel_lanes(parallel_same_dir, ego_key)

{(5, 2): [(4, 2, 62, (2,)), (7, 2, 100, (-1, -2))],
 (5, 1): [(8, -1, 104, (-1,)), (4, 1, 62, (1,))],
 (4, -1): [(5, -1, 63, (-1,)), (7, 1, 80, (-1,))],
 (4, -2): [(5, -2, 63, (-2,)), (8, -2, 92, (-1, -2))],
 (7, -2): [(4, 2, 84, (2, 1)), (8, -2, 126, (-2,))],
 (8, 1): [(7, 1, 125, (1,)), (4, 1, 88, (1,))],
 (7, -1): [(5, -1, 96, (1,)), (8, -1, 126, (-1,))],
 (8, 2): [(7, 2, 125, (2,)), (5, -2, 108, (2, 1))]}

In [23]:
wps = get_lane_waypoints(all_waypoints, intersecting_right[1], road_lane_to_orientation)
ls = convert_to_shapely_linestring(wps)

wps1 = get_lane_waypoints(all_waypoints, ego_lane[0], road_lane_to_orientation)
ls2 = convert_to_shapely_linestring(wps1) 

[[(7, -2), (126, -2), (8, -2)], [(7, -1), (126, -1), (8, -1)]]

In [25]:
for element in parallel_same_dir:
    for item in element:
        draw(item[0], item[1])

In [1]:
# if road_id, lane_id belongs to parallel_same_dir please pass in same_dir = True, vice versa
# usage application:
#     for key in list(incoming_road_lane_id_to_outgoing_lane_id_dict.keys()):
#         get_direction_information(key[0], key[1], ...)
        
def get_direction_information(road_id, lane_id, ego_info, in_out_dict, same_dir, intersecting_left, intersecting_right, parallel_same_dir, parallel_opposite_dir):
    draw(road_id, lane_id)
    right_turning_lane = False
    left_turning_lane = False
    right_most_turning_lane = False
    left_to_the_current = False
    right_next_to_the_current = False
    lanes_to_right = intersecting_right if same_dir else intersecting_left
    lanes_to_left = intersecting_left if same_dir else intersecting_right

    for connection in in_out_dict[(road_id, lane_id)]:
        print((connection[0], connection[1]))
        for right_lanes in lanes_to_right:
            if (connection[0], connection[1]) in right_lanes:
                right_turning_lane = True 

        for left_lanes in lanes_to_left:
            if (connection[0], connection[1]) in left_lanes:
                left_turning_lane = True
    
    # determine if the lane is right_most
    lane_group = parallel_same_dir if same_dir else parallel_opposite_dir
    maximum_id_magnitude = 0
    for lane in lane_group:
        maximum_id_magnitude = max(abs(lane[0][1]), maximum_id_magnitude)
    right_most_turning_lane = maximum_id_magnitude == abs(lane_id)
    
    ego_lane = ego_info[1]
    ego_road = [parallel_same_dir[0][0][0], parallel_same_dir[0][1][0], parallel_same_dir[0][2][0]]
    if (road_id in ego_road): 
        if not same_dir: 
            left_to_the_current = True # which could means right_to_the_current = False
            #since this function should only be applied on (road_id, lane_id) in in_out_dict.keys()
            # update all the fields in connecting lanes if needed
            # update_field_in_group(road_id, lane_id, parallel_opposite_dir, left_to_the_current, True)
            if (abs(ego_lane) == 1 and lane_id * ego_lane == 1):
                right_next_to_the_current = True 
                # update_field_in_group(road_id, lane_id, parallel_opposite_dir, right_next_to_the_current, True)

        else:
            if abs(lane_id) < abs(ego_lane):
                left_to_the_current = True #on the same road but with opposite direction
                # update_field_in_group(road_id, lane_id, parallel_opposite_dir, left_to_the_current, True)
            if (abs(ego_lane - lane_id) == 1):
                right_next_to_the_current = True
                # update_field_in_group(road_id, lane_id, parallel_opposite_dir, right_next_to_the_current, True)

            
    
    assert(right_turning_lane != left_turning_lane)
    print("right_turning_lane: ", right_turning_lane)
    print("left_turning_lane: ", left_turning_lane)
    print("left_to_the_ego_for_all_the_current_lanes: ", left_to_the_current)
    print("right_next_to_the_ego_for_all_the_current_lanes: ", right_next_to_the_current)

In [7]:
ego_nearest_waypoint = client.get_world().get_map().get_waypoint(
    ego_vehicle.get_location(), project_to_road=True
)
client.get_world().debug.draw_string(
    ego_nearest_waypoint.transform.location,
    "O",
    draw_shadow=False,
    color=carla.Color(r=255, g=0, b=0),
    life_time=100,
)

In [10]:
def get_lane_waypoints(all_waypoints, road_lane_collection, road_lane_to_orientation):
    
    incoming_road = road_lane_collection[0][0]
    incoming_lane = road_lane_collection[0][1]
    
    connecting_road = road_lane_collection[1][0]
    connecting_lane = road_lane_collection[1][1]
    
    outgoing_road = road_lane_collection[2][0]
    outgoing_lane = road_lane_collection[2][1]
    
    incoming_waypoints = filter_waypoints(all_waypoints, incoming_road, incoming_lane)
    connecting_waypoints = filter_waypoints(all_waypoints, connecting_road, connecting_lane)
    outgoing_waypoints = filter_waypoints(all_waypoints, outgoing_road, outgoing_lane)
    
    if(road_lane_to_orientation[(incoming_road, incoming_lane)][-1] == 0): # 0 : Starting from near junction
        incoming_waypoints = incoming_waypoints[::-1]
        
    if(road_lane_to_orientation[(outgoing_road, outgoing_lane)][-1] == 1): # 1 : Ending at junction
        outgoing_waypoints = outgoing_waypoints[::-1]


    first_connecting_waypoint = connecting_waypoints[0]
    last_connecting_waypoint = connecting_waypoints[-1]
    
    last_incoming_waypoint = incoming_waypoints[-1]
    
    dist1 = first_connecting_waypoint.transform.location.distance(last_incoming_waypoint.transform.location)
    dist2 = last_connecting_waypoint.transform.location.distance(last_incoming_waypoint.transform.location)

    if(dist1 > dist2):
        connecting_waypoints = connecting_waypoints[::-1]
        
    
    return incoming_waypoints + connecting_waypoints[1:-1] + outgoing_waypoints
    


    
def convert_to_shapely_linestring(waypoints):
    
    list_of_coordinates = [Point(waypoint.transform.location.x, waypoint.transform.location.y) for waypoint in waypoints]
    linestring = LineString(list_of_coordinates)
    
    return linestring


def separate_ego_lane_and_other_parallel_lanes(parallel_same_dir, ego_key):
    
    ego_lane = []
    other_lanes = []
    
    for elem in parallel_same_dir:
        flag = 0
        for item in elem:
            if(item == ego_key):
                flag = 1
        if(flag == 1):
            ego_lane.append(elem)
        else:
            other_lanes.append(elem)
    return ego_lane, other_lanes
            
    
    